In [ ]:
# -------------------------- Python & library version --------------------------
# Python version: 3.8.8 (default, Apr 13 2021, 15:08:03) [MSC v.1916 64 bit (AMD64)]
# pandas version: 1.2.4
# numpy version: 1.19.2
# matplotlib version: 3.3.4
# tqdm version: 4.59.0
# sktime version: 0.6.1
# xgboost version: 1.2.1
# seaborn version: 0.11.1
# scikit-learn version: 0.24.2
# ------------------------------------------------------------------------------

In [2]:
import sys
import sktime
import tqdm as tq
import xgboost as xgb
import matplotlib
import seaborn as sns
import sklearn as skl
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
print("-------------------------- Python & library version --------------------------")
print("Python version: {}".format(sys.version))
print("pandas version: {}".format(pd.__version__))
print("numpy version: {}".format(np.__version__))
print("matplotlib version: {}".format(matplotlib.__version__))
print("tqdm version: {}".format(tq.__version__))
print("sktime version: {}".format(sktime.__version__))
print("xgboost version: {}".format(xgb.__version__))
print("seaborn version: {}".format(sns.__version__))
print("scikit-learn version: {}".format(skl.__version__))
print("------------------------------------------------------------------------------")

-------------------------- Python & library version --------------------------
Python version: 3.8.8 (default, Apr 13 2021, 15:08:03) [MSC v.1916 64 bit (AMD64)]
pandas version: 1.2.4
numpy version: 1.19.2
matplotlib version: 3.3.4
tqdm version: 4.59.0
sktime version: 0.6.1
xgboost version: 1.2.1
seaborn version: 0.11.1
scikit-learn version: 0.24.2
------------------------------------------------------------------------------


In [3]:
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from sktime.forecasting.model_selection import temporal_train_test_split
from sktime.utils.plotting import plot_series
from xgboost import XGBRegressor

pd.set_option('display.max_columns', 30)

In [4]:
train = pd.read_csv('../data/preproc3_train.csv')

In [5]:
train.drop('Time', axis=1, inplace=True)

In [6]:
train.columns

Index(['building_id', 'sin_time', 'cos_time', 'day', 'month', 'day_hour_mean',
       'hour_mean', 'hour_std', 'holiday', 'Temperature', 'Precipitation',
       'Wind speed', 'Humidity', 'DI', 'Solar radiation', 'consumption',
       'CDH'],
      dtype='object')

In [ ]:
# out_dim = 1

# class Regressor(torch.nn.Module):
#     def __init__(self, input_size, em_size=16, output_size=12, a=10):
#         super().__init__()
#         self.linear = nn.Sequential(
#             nn.Linear(input_size, em_size),
#             nn.ReLU(),
#             nn.Linear(em_size, output_size),
#             nn.Softmax(),
#         )
#         self.reg = XGBRegressor(n_estimators=a, tree_method='gpu_hist', objective = 'multi:softprob')

#     def forward(self, x, y, train=True):
#         x = self.linear(x)
        
#         if train:
#           self.reg.fit(x.detach().cpu().numpy(), y.detach().cpu().numpy())
          
#         out = self.reg.predict(x.detach().cpu().numpy())
#         out = torch.tensor(out)

#         return preds

In [ ]:
# # train embedding models
# t_input = len(train.columns) - 1
# s_input = len(train.columns) - 2
# output_size = 15
# epochs = 100

# t_embedding = EmbeddingModel(t_input)
# s_embedding = EmbeddingModel(s_input)

# criterion = nn.CrossEntropyLoss()
# optimizer = nn.optim.Adam(s_embedding.params)

In [7]:
# Define SMAPE loss function
def SMAPE(true, pred):
    return np.mean((np.abs(true-pred))/(np.abs(true) + np.abs(pred))) * 100

In [19]:
#### alpha를 argument로 받는 함수로 실제 objective function을 wrapping하여 alpha값을 쉽게 조정할 수 있도록 작성했습니다.
# custom objective function for forcing model not to underestimate
def weighted_mse(alpha = 1):
    def weighted_mse_fixed(label, pred):
        residual = (label - pred).astype("float")
        grad = np.where(residual>0, -2*alpha*residual, -2*residual)
        hess = np.where(residual>0, 2*alpha, 2.0)
        return grad, hess
    return weighted_mse_fixed

def log_mae(label, pred):
    ae = np.log1p(label) - np.log1p(pred)
    grad = -ae / (label * abs(ae))
    hess = ae / (label ** 2 * abs(ae))
    return grad, hess

def rmsle_loss(y_pred, y_true):
    return 'rmsle', np.sqrt(np.mean((np.log1p(y_pred) - np.log1p(y_true))**2))

In [25]:
## gridsearchCV for best model : 대략 1시간 소요

from sklearn.model_selection import PredefinedSplit, GridSearchCV
from sklearn.metrics import make_scorer

smape = make_scorer(SMAPE, greater_is_better = False)

df = pd.DataFrame(columns = ['n_estimators', 'eta', 'min_child_weight','max_depth', 'colsample_bytree', 'subsample'])
preds = np.array([])

grid = {'n_estimators' : [200], 'eta' : [0.01], 'min_child_weight' : np.arange(1, 8, 1), 
        'max_depth' : np.arange(3,9,1) , 'colsample_bytree' :np.arange(0.8, 1.0, 0.1), 
        'subsample' :np.arange(0.8, 1.0, 0.1)} # fix the n_estimators & eta(learning rate)
        
for i in tqdm(np.arange(1, 101)):
    y = train.loc[train.building_id == i, 'consumption']
    x = train.loc[train.building_id == i, ].drop('consumption', axis=1)
    y_train, y_test, x_train, x_test = temporal_train_test_split(y = y, X = x, test_size = 168)
    
    
    pds = PredefinedSplit(np.append(-np.ones(len(x_train)-168), np.zeros(168)))
    gcv = GridSearchCV(estimator = XGBRegressor(seed = 0, gpu_id = 0, 
                                                tree_method = 'gpu_hist', predictor= 'gpu_predictor'),
                       param_grid = grid, scoring = smape, cv=pds, refit = True, verbose = True)
    
    
    gcv.fit(x_train, y_train)
    best = gcv.best_estimator_
    params = gcv.best_params_
    print(params)
    pred = best.predict(x_test)
    print(f'Building {i} SMAPE : {SMAPE(y_test, pred)}')
    preds = np.append(preds, pred)
    df = pd.concat([df, pd.DataFrame(params, index = [0])], axis = 0)
df.to_csv('./hyperparameter_teacher1.csv', index = False) # save the tuned parameters


  0%|          | 0/100 [00:00<?, ?it/s]

Fitting 1 folds for each of 168 candidates, totalling 168 fits
{'colsample_bytree': 0.9, 'eta': 0.01, 'max_depth': 4, 'min_child_weight': 1, 'n_estimators': 200, 'subsample': 0.9}
Building 1 SMAPE : 8.153339110837528
Fitting 1 folds for each of 168 candidates, totalling 168 fits


In [ ]:
xgb_params = pd.read_csv('./hyperparameter_teacher1.csv')

### find the bset iteration (given alpha = 100)

In [ ]:
scores = []   # smape 값을 저장할 list
best_it = []  # best interation을 저장할 list
for i in tqdm(range(100)):
    y = train.loc[train.building_id == i+1, 'consumption']
    x = train.loc[train.building_id == i+1, ].drop('consumption', axis=1)
    y_train, y_valid, x_train, x_valid = temporal_train_test_split(y = y, X = x, test_size = 168)
    
    xgb_reg = XGBRegressor(n_estimators = 20000, eta = 0.01, min_child_weight = xgb_params.iloc[i, 2],
                           max_depth = xgb_params.iloc[i, 3], colsample_bytree = xgb_params.iloc[i, 4], 
                           subsample = xgb_params.iloc[i, 5], seed=0,
                           tree_method = 'gpu_hist', predictor= 'gpu_predictor')
    # xgb_reg.set_params(**{'objective':weighted_mse(100)}) # alpha = 100으로 고정
    xgb_reg.set_params(**{'objective':'reg:squaredlogerror'}) # SMSLE
    xgb_reg.fit(x_train, y_train, eval_set=[(x_train, y_train), 
                                            (x_valid, y_valid)], early_stopping_rounds=3000, verbose=False)
    y_pred = xgb_reg.predict(x_valid)
    pred = pd.Series(y_pred)   
    
    sm = SMAPE(y_valid, y_pred)
    scores.append(sm)
    # print(xgb_reg.best_iteration, sm)
    best_it.append(xgb_reg.best_iteration) ## 실제 best iteration은 이 값에 +1 해주어야 함.

### alpha tuning for weighted MSE

In [ ]:
alpha_list = []
smape_list = []
for i in tqdm(range(100)):
    y = train.loc[train.building_id == i+1, 'consumption']
    x = train.loc[train.building_id == i+1, ].drop('consumption', axis=1)
    y_train, y_test, x_train, x_test = temporal_train_test_split(y = y, X = x, test_size = 168)
    xgb = XGBRegressor(seed = 0,
                      n_estimators = best_it[i], eta = 0.01, min_child_weight = xgb_params.iloc[i, 2],
                      max_depth = xgb_params.iloc[i, 3], colsample_bytree = xgb_params.iloc[i, 4], subsample = xgb_params.iloc[i, 5],
                      tree_method = 'gpu_hist', predictor= 'gpu_predictor')
    
    xgb.fit(x_train, y_train)
    pred0 = xgb.predict(x_test)
    best_alpha = 0
    score0 = SMAPE(y_test,pred0)
    
    for j in [1, 3, 5, 7, 10, 25, 50, 75, 100]:
        xgb = XGBRegressor(seed = 0,
                      n_estimators = best_it[i], eta = 0.01, min_child_weight = xgb_params.iloc[i, 2],
                      max_depth = xgb_params.iloc[i, 3], colsample_bytree = xgb_params.iloc[i, 4], subsample = xgb_params.iloc[i, 5],
                      tree_method = 'gpu_hist', predictor= 'gpu_predictor')
        xgb.set_params(**{'objective' : weighted_mse(j)})
    
        xgb.fit(x_train, y_train)
        pred1 = xgb.predict(x_test)
        score1 = SMAPE(y_test, pred1)
        if score1 < score0:
            best_alpha = j
            score0 = score1
    
    alpha_list.append(best_alpha)
    smape_list.append(score0)
    print("building {} || best score : {} || alpha : {}".format(i+1, score0, best_alpha))

In [ ]:
no_df = pd.DataFrame({'score':smape_list})
plt.bar(np.arange(len(no_df))+1, no_df['score'])
plt.plot([1,60], [10, 10], color = 'red')

## 4. test inference

### preprocessing for test data

In [ ]:
xgb_params['alpha'] = alpha_list
xgb_params['best_it'] = best_it
xgb_params.head()

In [ ]:
xgb_params.to_csv('./hyperparameter_xgb_final.csv', index=False)

In [ ]:
## best hyperparameters 불러오기
xgb_params = pd.read_csv('./hyperparameter_xgb_final.csv')
xgb_params.head()

In [ ]:
best_it = xgb_params['best_it'].to_list()
best_it[0]        # 1051

### seed ensemble
#### - seed별로 예측값이 조금씩 바뀝니다. 
#### - seed의 영향을 제거하기 위해 6개의 seed(0부터 5)별로 훈련, 예측하여 6개 예측값의 평균을 구했습니다.

In [ ]:
test = pd.read_csv('../data/preproc3_test.csv')
test = test.drop('Time', axis=1)
# test = test.drop('cluster', axis=1)
# test = test.drop('type', axis=1)

preds = np.array([]) 
for i in tqdm(range(100)):
    
    pred_df = pd.DataFrame()   # 시드별 예측값을 담을 data frame
    
    for seed in range(5): # 각 시드별 예측
        y_train = train.loc[train.building_id == i+1, 'consumption']
        x_train, x_test = train.loc[train.building_id == i+1, ].drop('consumption', axis=1), test.loc[test.building_id == i+1, ]
        x_test = x_test[x_train.columns]
        
        xgb = XGBRegressor(seed = seed, n_estimators = best_it[i], eta = 0.01, 
                           min_child_weight = xgb_params.iloc[i, 2], max_depth = xgb_params.iloc[i, 3], 
                           colsample_bytree=xgb_params.iloc[i, 4], subsample=xgb_params.iloc[i, 5],
                           tree_method = 'gpu_hist', predictor= 'gpu_predictor')
    
        if xgb_params.iloc[i,6] != 0:  # 만약 alpha가 0이 아니면 weighted_mse 사용
            xgb.set_params(**{'objective':weighted_mse(xgb_params.iloc[i,6])})
        
        xgb.fit(x_train, y_train)
        y_pred = xgb.predict(x_test)
        pred_df.loc[:,seed] = y_pred   # 각 시드별 예측 담기
        
    pred = pred_df.mean(axis=1)        # (i+1)번째 건물의 예측 =  (i+1)번째 건물의 각 시드별 예측 평균값
    preds = np.append(preds, pred)   

In [ ]:
preds = pd.Series(preds)

fig, ax = plt.subplots(100, 1, figsize=(100,200), sharex = True)
ax = ax.flatten()
for i in range(100):
    train_y = train.loc[train.building_id == i+1, 'consumption'].reset_index(drop = True)
    test_y = preds[i*168:(i+1)*168]
    ax[i].scatter(np.arange(2040) , train.loc[train.building_id == i+1, 'consumption'])
    ax[i].scatter(np.arange(2040, 2040+168) , test_y)
    ax[i].tick_params(axis='both', which='major', labelsize=6)
    ax[i].tick_params(axis='both', which='minor', labelsize=4)
#plt.savefig('./predict_xgb.png')
plt.show()

In [ ]:
submission = pd.read_csv('../data/sample_submission.csv')
submission['answer'] = preds
submission.to_csv('../data/submission/submission_xgb_noclip.csv', index = False)

## 5. post processing

#### weighted mse와 같은 맥락에서, 과도한 underestimate를 막기 위해 예측값을 후처리했습니다.
##### - 예측 주로부터 직전 4주(train set 마지막 28일)의 건물별 요일별 시간대별 전력소비량의 최솟값을 구한 뒤, 
##### - test set의 같은 건물 요일 시간대의 예측값과 비교하여 만약 1번의 최솟값보다 예측값이 작다면 최솟값으로 예측값을 대체해주었습니다.
##### - public score 0.01 , private score 0.08 정도의 성능 향상이 있었습니다.

In [ ]:
train_to_post = pd.read_csv('../data/preproc_train.csv')

In [ ]:
train_to_post

In [ ]:
# cols = ['num', 'date_time', 'power', 'temp', 'wind','hum' ,'prec', 'sun', 'non_elec', 'solar']

# train_to_post.columns = cols
date = pd.to_datetime(train_to_post.Time)
train_to_post['hour'] = date.dt.hour
train_to_post['day'] = date.dt.weekday
train_to_post['month'] = date.dt.month
train_to_post['week'] = date.dt.weekofyear
train_to_post = train_to_post.loc[(('2022-08-17'>train_to_post.Time)|(train_to_post.Time>='2022-08-18')), ].reset_index(drop = True)

pred_clip = []
test_to_post = pd.read_csv('../data/preproc_test.csv',  encoding = 'cp949')
# cols = ['num', 'date_time', 'temp', 'wind','hum' ,'prec', 'sun', 'non_elec', 'solar']
# test_to_post.columns = cols
date = pd.to_datetime(test_to_post.Time)
test_to_post['hour'] = date.dt.hour
test_to_post['day'] = date.dt.weekday
test_to_post['month'] = date.dt.month
test_to_post['week'] = date.dt.weekofyear

## submission 불러오기
df = pd.read_csv('../data/submission/submission_xgb_noclip5.csv')
for i in range(100):
    min_data = train_to_post.loc[train_to_post.building_id == i+1, ].iloc[-28*24:, :] ## 건물별로 직전 28일의 데이터 불러오기
    ## 요일별, 시간대별 최솟값 계산
    min_data = pd.pivot_table(min_data, values = 'consumption', index = ['day', 'hour'], aggfunc = min).reset_index() 
    pred = df.answer[168*i:168*(i+1)].reset_index(drop=True) ## 168개 데이터, 즉 건물별 예측값 불러오기
    day =  test_to_post.day[168*i:168*(i+1)].reset_index(drop=True) ## 예측값 요일 불러오기
    hour = test_to_post.hour[168*i:168*(i+1)].reset_index(drop=True) ## 예측값 시간 불러오기
    df_pred = pd.concat([pred, day, hour], axis = 1)
    df_pred.columns = ['pred', 'day', 'hour']
    for j in range(len(df_pred)):
        min_consumption = min_data.loc[(min_data.day == df_pred.day[j])&(min_data.hour == df_pred.hour[j]), 'consumption'].values[0]
        if df_pred.pred[j] < min_consumption:
            pred_clip.append(min_consumption)
        else:
            pred_clip.append(df_pred.pred[j])

##### 초록색으로 표시된 값이 원래의 예측값, 주황색이 후처리된 예측값입니다.
##### 변동이 거의 없는 건물도 있으나, 유의미하게 바뀐 건물도 확인됩니다. 

In [ ]:
pred_origin = df.answer
pred_clip = pd.Series(pred_clip)

for i in range(100):
    power = train_to_post.loc[train_to_post.building_id == i+1, 'consumption'].reset_index(drop=True)
    preds = pred_clip[i*168:(i+1)*168]
    preds_origin = pred_origin[i*168:(i+1)*168]
    preds.index = range(power.index[-1], power.index[-1]+168)
    preds_origin.index = range(power.index[-1], power.index[-1]+168)
    
    plot_series(power, preds,  preds_origin, markers = [',', ',', ','])

#### create submission file 

In [ ]:
submission = pd.read_csv('../data/sample_submission.csv')
submission['answer'] = pred_clip
submission.to_csv('../data/submission/submission_xgb_final.csv', index = False)